In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests


In [2]:
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

def get_data(pageNo, keyword):  
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run Chrome in headless mode (no GUI)
    driver = webdriver.Chrome(options=chrome_options)

    url = f'https://www.amazon.in/s?k={keyword}&page={pageNo}'
    driver.get(url)

    # Wait for a certain element to be present (adjust as needed)
    driver.implicitly_wait(10)

    content = driver.page_source
    soup = BeautifulSoup(content, 'html.parser')

    alls = []
    unknown_count = 0  # Counter for Unknown-Product
    for d in soup.find_all('div', class_='sg-col-inner'):
        # Check the first class
        name_element = d.find('span', class_='a-size-medium a-color-base a-text-normal')
        
        # If the first class is not found, check the second class
        if not name_element:
            name_element = d.find('span', class_='a-size-base-plus a-color-base a-text-normal')

        # Skip if the name is not available
        if not name_element:
            continue

        # Extract the name
        name = name_element.text.strip()

        # Count Unknown-Product and skip the first 4
        if name == "Unknown-Product":
            unknown_count += 1
            if unknown_count <= 4:
                continue

        rating_element = d.find('span', attrs={'class': 'a-icon-alt'})
        rating = rating_element.text.strip() if rating_element else "0"

        users_rated_element = d.find('span', attrs={'class': 'a-size-base s-underline-text'})
        users_rated_text = users_rated_element.text.strip() if users_rated_element else "0"
        
        # Use a try-except block to handle the conversion safely
        try:
            users_rated = int(re.sub('[^0-9]', '', users_rated_text))
        except ValueError:
            users_rated = 0

        price_element = d.find('span', attrs={'class': 'a-price-whole'})
        
        price = "Rs "+price_element.text.strip() if price_element else "0"

        url_element = d.find('a', class_='a-link-normal s-no-outline')
        url = "https://www.amazon.in" + url_element['href'] if url_element else "Not Found"

        all1 = [name, rating, users_rated, price, url]
        alls.append(all1)

    driver.quit()

    return alls





In [3]:
import subprocess
import os

# Command to start the Node.js server in the background
command = "node server.js"

# Redirecting stdout and stderr to /dev/null or NUL
with open(os.devnull, 'w') as DEVNULL:
    try:
        subprocess.Popen(command, shell=True, stdout=DEVNULL, stderr=DEVNULL)
    except subprocess.CalledProcessError as e:
        print("Error:", e)

In [4]:
import webbrowser

# Path to your index2.html file
html_file_path = "index2.html"  # Replace this with the actual path to your index2.html file

# Open index2.html file in the default web browser
webbrowser.open_new_tab(html_file_path)

True

In [5]:
import json
import time
import pandas as pd  # Import pandas module at the beginning
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler


In [6]:
import subprocess

# Global variable to store the previous state of the data
previous_data = {}

# Define a function to handle file system events
class MyHandler(FileSystemEventHandler):
    def on_modified(self, event):
        global previous_data
        
        # Read data from data.json file
        with open('data.json', 'r') as file:
            data = json.load(file)

        # Compare current data with previous data
        if data != previous_data:
            # Extract values
            keyword = data['name']
            pageno = data['page']

            # Rerun your function if keyword or page number changes
            
            # Your function that reruns when keyword or page number changes
            results = []
            for i in range(1, int(pageno)+1):
                results.append(get_data(i,keyword))
            flatten = lambda l: [item for sublist in l for item in sublist]
            df = pd.DataFrame(flatten(results),columns=['Product Name','Rating','Customers_Rated', 'Price', 'URL'])
            df.to_csv('amazon_products.csv', index=False, encoding='utf-8')
            df = pd.read_csv("amazon_products.csv")
            try:
                subprocess.run(["python", "script.py"], check=True)
            except subprocess.CalledProcessError as e:
                print(f"Error: {e}")
            # Update previous_data with current data
            previous_data.update(data)

# Create an observer and attach the handler
event_handler = MyHandler()
observer = Observer()
observer.schedule(event_handler, path='.', recursive=False)
observer.start()

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    observer.stop()
observer.join()


Error: Command '['python', 'script.py']' returned non-zero exit status 2.
Error: Command '['python', 'script.py']' returned non-zero exit status 2.
